In [5]:
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.agents import AssistantAgent
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_agentchat.conditions import MaxMessageTermination, TextMentionTermination
from autogen_agentchat.ui import Console

In [6]:
model = OpenAIChatCompletionClient(model="gpt-4o-mini")

clarity_agent = AssistantAgent(
    "ClarityAgent",
    model_client=model,
    system_message="""당신은 명확성과 간결성에 중점을 둔 전문 편집자입니다.
            당신의 임무는 모호함과 중복을 제거하고 모든 문장을 명확하고 명확하게 만드는 것입니다.
            설득력이나 어조는 걱정하지 마세요. 메시지를 읽고 이해하기 쉽게 만드세요.""",
)

tone_agent = AssistantAgent(
    "ToneAgent",
    model_client=model,
    system_message="""당신은 감정적인 톤과 전문성에 중점을 둔 커뮤니케이션 코치입니다.
            당신의 임무는 이메일을 따뜻하고, 자신감 넘치고, 인간적으로 들리게 하는 동시에 전문적이고
            청중에게 적합한 방식으로 전달하는 것입니다. 감정적인 공감을 높이고, 표현을 다듬고,
            경직되거나 차갑거나 지나치게 캐주얼하게 느껴질 수 있는 단어들을 수정하세요.""",
)

persuasion_agent = AssistantAgent(
    "PersuasionAgent",
    model_client=model,
    system_message="""당신은 마케팅, 행동 심리학, 그리고 카피라이팅을 전공한 설득 전문가입니다.
            당신의 임무는 이메일의 설득력을 강화하는 것입니다. 행동 촉구를 개선하고, 주장을 체계적으로 구성하고, 이점을 강조하세요. 약하거나 수동적인 언어는 제거하세요.""",
)

synthesizer_agent = AssistantAgent(
    "SynthesizerAgent",
    model_client=model,
    system_message="""귀하는 고급 이메일 작성 전문가입니다. 귀하의 역할은 모든 이전 상담원의 답변과 수정 사항을 읽고, **최고의 아이디어**를 종합하여 통합적이고 세련된 이메일 초안을 작성하는 것입니다.
            다음 사항에 중점을 둡니다. 명확성, 어조, 설득력 향상;
            일관성, 유창성, 자연스러운 어조 확보; 전문적이고 효과적이며 읽기 쉬운 이메일 작성.""",
)

critic_agent = AssistantAgent(
    "CriticAgent",
    model_client=model,
    system_message="""이메일 품질 평가자입니다. 합성된 이메일을 최종 검토하고
            전문적인 기준을 충족하는지 확인하는 것이 귀하의 임무입니다. 이메일 검토 항목:
            명확성과 흐름, 적절한 전문적인 어조, 효과적인 행동 촉구, 그리고 전반적인 일관성.
            건설적이면서도 단호한 태도를 유지하십시오. 이메일에 중대한 결함(불분명한 메시지, 비전문적인 어조,
            또는 핵심 요소 누락)이 있는 경우, 구체적인 개선 제안을 하나 제시하십시오. 이메일이 전문적인 기준을 충족하고 효과적으로 소통하는 경우, 
            "이 이메일은 전문적인 기준을 충족합니다."라고 답장한 후 새 줄에 `TERMINATE`를 입력하십시오. 전문적인 용도로 사용하기에 충분히 완벽한 이메일만 승인해야 하며, 타협하지 마십시오.""",
)

In [7]:
text_termination = TextMentionTermination("TERMINATE")
max_messages_termination = MaxMessageTermination(max_messages=30)

termination_condition = text_termination | max_messages_termination

In [8]:
team = RoundRobinGroupChat(
    participants=[
        clarity_agent,
        tone_agent,
        persuasion_agent,
        synthesizer_agent,
        critic_agent,
    ],
    termination_condition=termination_condition,
)

await Console(
    team.run_stream(
        task="씨큐브 Poc 관련 Test 결과보고서 요약본 첨부하여 공유 드립니다."
    )
)


---------- TextMessage (user) ----------
씨큐브 Poc 관련 Test 결과보고서 요약본 첨부하여 공유 드립니다.
---------- TextMessage (ClarityAgent) ----------
씨큐브 PoC 관련 테스트 결과 보고서 요약본을 첨부하여 공유합니다.
---------- TextMessage (ToneAgent) ----------
안녕하세요,

씨큐브 PoC 관련 테스트 결과 보고서의 요약본을 첨부하여 공유 드립니다. 이 자료가 도움이 되길 바라며, 궁금한 점이나 추가로 논의하고 싶은 사항이 있다면 언제든지 편하게 말씀해 주세요.

감사합니다.
---------- TextMessage (PersuasionAgent) ----------
안녕하세요,

씨큐브 PoC 테스트 결과 보고서의 요약본을 첨부하였습니다. 이 자료가 여러분에게 명확한 인사이트를 제공할 것입니다. 불확실한 부분이나 논의가 필요하신 사항이 있다면 주저하지 말고 언제든지 연락해 주세요. 여러분의 의견을 듣고 함께 발전할 수 있는 기회를 기대합니다.

감사합니다.
---------- TextMessage (SynthesizerAgent) ----------
제목: 씨큐브 PoC 테스트 결과 보고서 요약본 공유

안녕하세요,

씨큐브 PoC 테스트 결과 보고서의 요약본을 첨부하여 공유드립니다. 이 자료가 여러분에게 유용한 인사이트를 제공하길 바랍니다. 

혹시 자료에 대해 궁금한 점이 있거나 추가적인 논의가 필요하신 사항이 있다면 언제든지 편하게 연락해 주세요. 여러분의 의견을 듣고 함께 발전할 수 있는 기회를 기대합니다.

감사합니다.

[귀하의 이름]  
[귀하의 직함]  
[귀하의 연락처]  
[귀하의 회사명]  
---------- TextMessage (CriticAgent) ----------
이 이메일은 전문적인 기준을 충족합니다.  
TERMINATE


TaskResult(messages=[TextMessage(id='034e91d3-6760-4515-8cb0-a2a0b50996f7', source='user', models_usage=None, metadata={}, created_at=datetime.datetime(2025, 10, 20, 8, 10, 21, 170039, tzinfo=datetime.timezone.utc), content='씨큐브 Poc 관련 Test 결과보고서 요약본 첨부하여 공유 드립니다.', type='TextMessage'), TextMessage(id='f86b289d-b17a-4ce0-bee5-db6ba79f0557', source='ClarityAgent', models_usage=RequestUsage(prompt_tokens=105, completion_tokens=21), metadata={}, created_at=datetime.datetime(2025, 10, 20, 8, 10, 21, 861830, tzinfo=datetime.timezone.utc), content='씨큐브 PoC 관련 테스트 결과 보고서 요약본을 첨부하여 공유합니다.', type='TextMessage'), TextMessage(id='c6f98164-eb23-43af-813e-892097ea3bf1', source='ToneAgent', models_usage=RequestUsage(prompt_tokens=168, completion_tokens=63), metadata={}, created_at=datetime.datetime(2025, 10, 20, 8, 10, 23, 9454, tzinfo=datetime.timezone.utc), content='안녕하세요,\n\n씨큐브 PoC 관련 테스트 결과 보고서의 요약본을 첨부하여 공유 드립니다. 이 자료가 도움이 되길 바라며, 궁금한 점이나 추가로 논의하고 싶은 사항이 있다면 언제든지 편하게 말씀해 주세요.\n\n감사합니다.', type=